### Testing out Hyperspace on MNIST Data - Model Resiliency Test

In [ ]:
from hyperspace import create_hyperspace
from ray import tune
import tensorflow as tf
from torch import nn
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers
from ray.tune.suggest.skopt import SkOptSearch
from skopt import Optimizer
import ray
from tqdm import tqdm
import torch
import torchvision
import statistics
import pandas as pd

In [ ]:
ray.init()

### Tensorflow Model Objective Function Definition

In [10]:
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [8]:
def mnist_tf_objective(config):
    mnist = tf.keras.datasets.mnist

    (x_train, y_train),(x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dropout(config['dropout']),
      tf.keras.layers.Dense(10, activation='softmax')
    ])

    opt = tf.keras.optimizers.Adam(learning_rate=config['learning_rate'])

    model.compile(optimizer=opt,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    res = model.fit(x_train, y_train, epochs=config['epochs'], batch_size=config['batch_size'])
    res_test = model.evaluate(x_test, y_test)
    # res test[0] reports the loss from the evaluation, res_test[1] reports the accuracy
    tune.report(test_loss = res_test[0])
    return res_test[0]

In [ ]:
### Defining the hyperspace
hyperparameters = [(0.00001, 0.1),  # learning_rate
                   (0.2, 0.9),  # dropout
                   (10, 100),  # epochs 
                   (10, 1000)]  # batch size
space = create_hyperspace(hyperparameters)

In [ ]:
space

#### Run hypertune for Tensorflow Model...

In [ ]:
%%capture tf_run_output

### for each space in hyperspace, we want to search the space using ray tune
results = []
for section in tqdm(space):
    # create a skopt gp minimize object
    optimizer = Optimizer(section)
    search_algo = SkOptSearch(optimizer, ['learning_rate', 'dropout', 'epochs', 'batch_size'],
                              metric='test_loss', mode='min')
    # not using a gpu because running on local
    analysis = tune.run(mnist_tf_objective, search_alg=search_algo, num_samples=20, local_dir="~/Documents/hyper_resilient/experiments/exp1")
    results.append(analysis)

# # print out the best result
# i = 0
# for a in results:
#     print("Best config for space "+str(i)+": "+a.get_best_config(metric="test_loss", mode="min"))
#     i +=1

In [ ]:
tf_results = results

In [ ]:
tf_results

In [ ]:
all_tf_results = tf_results[0].results_df
for i in range(1, len(tf_results)):
    all_tf_results = all_tf_results.append(tf_results[i].results_df)

In [ ]:
all_tf_results.to_csv('full_tf_results.csv')

### Pytorch Model Objective Function Definition

In [26]:
class NumberNet(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.model = nn.Sequential(
            nn.Flatten(), 
            nn.Linear(784, 128), 
            nn.ReLU(), 
            nn.Dropout(config['dropout']), 
            nn.Linear(128, 10))
        self.criterion = nn.CrossEntropyLoss()
        self.config = config
        self.test_loss = None
        self.accuracy = pl.metrics.classification.Accuracy()
    
    def train_dataloader(self):
        return torch.utils.data.DataLoader(torchvision.datasets.MNIST("~/resiliency/", train=True, 
                                                                      transform=torchvision.transforms.ToTensor(), target_transform=None, download=True), 
                                           batch_size=int(self.config['batch_size']))
    
    def test_dataloader(self):
        return torch.utils.data.DataLoader(torchvision.datasets.MNIST("~/resiliency/", train=True, 
                                                                      transform=torchvision.transforms.ToTensor(), target_transform=None, download=True), 
                                           batch_size=int(self.config['batch_size']))
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.config['learning_rate'])
        return optimizer
    
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        logits = self.forward(x)
        loss = self.criterion(logits, y)
        logs = {'train_loss': loss}
        return {'loss': loss}
    
    def test_step(self, test_batch, batch_idx):
        x, y = test_batch
        logits = self.forward(x)
        loss = self.criterion(logits, y)
        accuracy = self.accuracy(logits, y)
        logs = {'test_loss': loss, 'test_accuracy': accuracy}
        return {'test_loss': loss, 'logs': logs, 'test_accuracy': accuracy}
    
    def test_epoch_end(self, outputs):
        loss = []
        for x in outputs:
            loss.append(float(x['test_loss']))
        avg_loss = statistics.mean(loss)
        tensorboard_logs = {'test_loss': avg_loss}
        self.test_loss = avg_loss
        accuracy = []
        for x in outputs:
            accuracy.append(float(x['test_accuracy']))
        avg_accuracy = statistics.mean(accuracy)
#         print("Self.accuracy.compute()")
#         print(self.accuracy.compute())
        self.test_accuracy = avg_accuracy
        return {'avg_test_loss': avg_loss, 'log': tensorboard_logs, 'avg_test_accuracy': avg_accuracy}


In [4]:
def mnist_pt_objective(config):
    model = NumberNet(config)
    trainer = pl.Trainer(max_epochs=config['epochs'])
    trainer.fit(model)
    trainer.test(model)
    tune.report(test_loss=model.test_loss)
    return model.test_loss

In [27]:
### Experimenting with the pytorch model to figure out the accuracy issue
model = NumberNet({'learning_rate': .001, 'dropout': 0.2, 'batch_size': 64, 'epochs': 25})
trainer = pl.Trainer(max_epochs=2)
trainer.fit(model)
trainer.test(model)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | Sequential       | 101 K 
1 | criterion | CrossEntropyLoss | 0     
2 | accuracy  | Accuracy         | 0     


Epoch 1: 100%|██████████| 938/938 [00:14<00:00, 65.19it/s, loss=0.127, v_num=39]

Saving latest checkpoint..


Testing:  99%|█████████▉| 931/938 [00:07<00:00, 117.18it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'avg_test_accuracy': 0.9629031026414209,
 'avg_test_loss': 0.12481893612859227,
 'test_loss': 0.12481893612859227}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 938/938 [00:08<00:00, 116.33it/s]


[{'test_loss': 0.12481893612859227,
  'avg_test_loss': 0.12481893612859227,
  'avg_test_accuracy': 0.9629031026414209}]

In [ ]:
%%capture pt_run_output
# hyperparameters = [(0.00000001, 0.1),  # learning_rate
#                    (0.0, 0.9),  # dropout
#                    (10, 100),  # epochs 
#                    (10, 1000)]  # batch size
# space = create_hyperspace(hyperparameters)

### for each space in hyperspace, we want to search the space using ray tune

results = []
for section in tqdm(space):
    # create a skopt gp minimize object
    optimizer = Optimizer(section)
    search_algo = SkOptSearch(optimizer, ['learning_rate', 'dropout', 'epochs', 'batch_size'],
                              metric='test_loss', mode='min')
    # not using a gpu because running on local
    analysis = tune.run(mnist_pt_objective, search_alg=search_algo, num_samples=20)
    results.append(analysis)

# print out the best result
# i = 0
# for a in results:
#     print("Best config for space "+str(i)+": "+a.get_best_config(metric="avg_test_loss", mode="min"))
#     i +=1

In [ ]:
pt_results = results

In [ ]:
pt_results

In [ ]:
all_pt_results = pt_results[0].results_df
for i in range(1, len(pt_results)):
    all_pt_results = all_pt_results.append(pt_results[i].results_df)

In [ ]:
all_pt_results

In [ ]:
just_pt_results = all_pt_results[['config.learning_rate','config.dropout', 'config.epochs', 'config.batch_size', 'test_loss']]